# Load and prepare data

In [1]:
import pandas as pd


test_passages_df = pd.read_json(
    "./data/simple-legal-questions-pl/passages.jl", lines=True
).set_index("_id").sort_index()

test_questions_df = pd.read_json(
    "./data/simple-legal-questions-pl/questions.jl", lines=True
).set_index("_id").sort_index()

test_answers_df = pd.read_json(
    "./data/simple-legal-questions-pl/answers.jl", lines=True
).set_index("question-id").sort_index()

test_relevant_df = pd.read_json(
    "./data/simple-legal-questions-pl/relevant.jl", lines=True
).set_index("question-id").sort_index()

In [2]:
test_passages_df.head()

,title,text
_id,,
1993_599_1,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 1. W ustawie z dnia 8 stycznia 1993 r. o ...
1993_599_2,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 2. W okresie od dnia wejścia w życie usta...
1993_599_3,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 3. Minister Finansów ogłosi w Dzienniku U...
1993_599_4,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 4. Ustawa wchodzi w życie z dniem 1 stycz...
1993_602_1,Ustawa z dnia 10 grudnia 1993 r. o zmianie nie...,Art. 1. W ustawie z dnia 29 maja 1974 r. o zao...


In [3]:
test_questions_df.head()

,text
_id,
1,"Czy żołnierz, który dopuszcza się czynnej napa..."
2,Z ilu osób składa się komisja przetargowa?
3,Do jakiej wysokości za zobowiązania spółki odp...
4,"Kiedy ustala się wartość majątku obrotowego, k..."
5,"Jakiej karze podlega armator, który wykonuje r..."


In [4]:
test_answers_df.head()

,score,answer
question-id,,
1,1.0,"Tak, podlega karze aresztu wojskowego albo poz..."
2,1.0,Komisja przetargowa składa się z co najmniej t...
3,1.0,Komandytariusz odpowiada za zobowiązania spółk...
4,1.0,Wartość rzeczowych składników majątku obrotowe...
5,1.0,Podlega karze pieniężnej do wysokości 1 000 00...


In [5]:
test_relevant_df.head()

,passage-id,score
question-id,,
1,1997_553_345,1
2,2004_177_21,1
3,1996_465_111,1
4,1994_591_35,1
5,2001_1441_74,1


In [6]:
print(f"Number of rows before fitter: {len(test_answers_df)}")

test_answers_df = test_answers_df[
    (test_answers_df["score"] == 0) | (test_answers_df["score"] == 1)
]

print(f"Number of rows after fitter: {len(test_answers_df)}")

Number of rows before fitter: 638
Number of rows after fitter: 636


In [7]:
import json


with open("./data/poquad-dev.json") as f:
    poquad_dev = json.load(f)["data"]

# Task 8
Use apohllo/plt5-base-poquad which was trained on PoQuAD.

In [8]:
from torch import no_grad, backends, device


if backends.mps.is_available():
    current_device = device("mps")
else:
    current_device = device("cpu")
    
print(f"Device is {current_device}")

Device is mps


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


class QuestionAnswering:
    def __init__(self, model_name: str, device) -> None:
        self._model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._device = device

    def answer(self, question: str, context: str, p_limit: float = 0.9, k_limit: int = 50) -> str:
        input_text = f"Pytanie: {question}. Kontekst: {context}"
        
        tokens = self._tokenizer.encode(
            input_text, 
            return_tensors="pt",
        ).to(self._device)

        with no_grad():
            outputs = self._model.generate(
                tokens, 
                max_length=1000, 
                do_sample=True, 
                top_p=p_limit, 
                top_k=k_limit,
            )

        output_text = self._tokenizer.decode(outputs[0], skip_special_tokens=True)

        return output_text

/Users/piotrbialy/Projects/agh-data-science/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model_name = "apohllo/plt5-base-poquad"

qa = QuestionAnswering(model_name, current_device)

In [11]:
_id = 2
_question = test_questions_df.loc[_id]["text"]
_context = test_passages_df.loc[test_relevant_df.loc[_id]["passage-id"]]["text"]

_answer = qa.answer(question=_question, context=_context)

print(f"question: {_question}")
print(f"context: {_context}")
print(f"answer: {_answer}")

question: Z ilu osób składa się komisja przetargowa?
context: Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zamawiającego. 2. Komisja przetargowa składa się z co najmniej trzech osób. 3. Kierownik zamawiającego określa organizację, skład, tryb pracy oraz zakres obowiązków członków komisji przetargowej, mając na celu zapewnienie sprawności jej działania, indywidualizacji odpowiedzialności jej członków za wykonywane czynności oraz przejrzystości jej prac. 4. Jeżeli dokonanie określonych czynności związanych z przygotowaniem i przeprowadzeniem postępowania o udzielenie zamówienia wymaga wiadomości specjalnych, kierownik zamawiającego, z własnej inicjatywy lub na wniosek komisji przetargowej, może powołać biegłych. Przepis art. 17 stosuje się.
answer: trzech


/Users/piotrbialy/Projects/agh-data-science/venv/lib/python3.11/site-packages/transformers/generation/logits_process.py:451: UserWarning: torch.topk support for k>16 by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Shape.mm:71.)
  indices_to_remove = scores < torch.topk(scores, top_k)[0][..., -1, None]
/Users/piotrbialy/Projects/agh-data-science/venv/lib/python3.11/site-packages/transformers/generation/logits_process.py:413: UserWarning: torch.sort is supported by MPS on MacOS 13+, please upgrade. Falling back to CPU (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Sort.mm:41.)
  sorted_logits, sorted_indices = torch.sort(scores, descending=False)
/Users/piotrbialy/Projects/agh-data-science/venv/lib/python3.11/site-packages/transformers/generation/logits_process.py:414: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade

# Task 9-10
Report the obtained performance of the model (in the form of a table). The report should include exact match and F1 score for the tokens appearing both in the reference and the predicted answer.

In [12]:
import re
import string
from collections import Counter


def normalize_answer(s: str) -> str:
    return " ".join(re.sub(f"[{string.punctuation}]", "", s.lower()).split())


def exact_match_score(prediction: str, truth: str) -> int:
    return int(normalize_answer(prediction) == normalize_answer(truth))


def f1_score(prediction: str, truth: str) -> float:
    def calculate_f1(precision, recall):
        if precision + recall == 0:
            return 0.0
        return (2 * precision * recall) / (precision + recall)

    pred_tokens = normalize_answer(prediction).split()
    truth_tokens = normalize_answer(truth).split()

    common = Counter(pred_tokens) & Counter(truth_tokens)
    num_same = sum(common.values())

    precision = num_same / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = num_same / len(truth_tokens) if len(truth_tokens) > 0 else 0

    return calculate_f1(precision, recall)

In [13]:
test_data = []

for qid, row in test_relevant_df.iterrows():
    try:
        item = (
            test_questions_df.loc[qid]["text"],
            test_passages_df.loc[test_relevant_df.loc[qid]["passage-id"]]["text"],
            test_answers_df.loc[qid]["answer"],
        ) 
        test_data.append(item)
    except:
        pass

In [14]:
from random import sample


validation_data = []

for article in poquad_dev:
    
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]

        for question_answer in paragraph["qas"]:
            question = question_answer["question"]

            answer = (
                question_answer["answers"][0]["generative_answer"] 
                if "answers" in question_answer 
                else ""
            )
            
            item = (question, context, answer)
            validation_data.append(item)

validation_data_sample = sample(validation_data, 4000)

In [15]:
from typing import Sequence
from tqdm import tqdm

import numpy as np


def eval_qa(qa: QuestionAnswering, data: Sequence[tuple[str, str, str]]) -> tuple[float, float]:
    exact_matches = []
    f1_scores = []

    for question, context, true_answer in tqdm(data):
        predicted_answer = qa.answer(question, context)

        exact_matches.append(exact_match_score(predicted_answer, true_answer))
        f1_scores.append(f1_score(predicted_answer, true_answer))

    return (
        np.mean(exact_matches, dtype=float),
        np.mean(f1_scores, dtype=float),
    )

In [16]:
test_exact_match, test_f1_score = eval_qa(qa, test_data)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 636/636 [10:47<00:00,  1.02s/it]


In [17]:
print(f"Scores for test data: \nExact match: {test_exact_match}\nF1 score: {test_f1_score}")

Scores for test data: 
Exact match: 0.24528301886792453
F1 score: 0.44090072893385934


In [18]:
validation_exact_match, validation_f1_score = eval_qa(qa, validation_data_sample)

100%|██████████| 4000/4000 [44:46<00:00,  1.49it/s]   


In [19]:
print(f"Scores for validation data: \nExact match: {validation_exact_match}\nF1 score: {validation_f1_score}")

Scores for validation data: 
Exact match: 0.42975
F1 score: 0.5745871322790022


# Task 11
Generate, report and analyze the answers for at least 10 questions provided by the best model on you test dataset.

In [20]:
def show_question(data: Sequence[tuple[str, str, str]], qa: QuestionAnswering, index: int) -> None:
    question, context, true_answer = data[index]
    predicted_answer = qa.answer(question, context)

    print(f"    Index: {index}")
    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True answer: {true_answer}")
    print(f"Predicted answer: {predicted_answer}")

In [21]:
indices = [0, 3, 120, 145, 150, 748, 1720, 1723, 1746, 1324]

In [22]:
for i in indices:
    show_question(validation_data, qa, i)

    Index: 0
Question: Czym są pisma rabiniczne?
Context: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik.
True answer: kompilacją poglądów różnych rabinów na określony temat
Predicted answer: kompilacją poglądów różnych rabinów na określony temat
    Index: 3
Question: W jakiej formie przekazana została Miszna?
Context: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch części

# Questions

## Does the performance on the validation dataset reflects the performance on your test set?

The performance on validation dataset is better than the performance for test dataset, especially in case of exact match score. This is due to the fact that test data are single domain specific.

## What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?

Most of answers are correct for selected questions. In case of question about Łukasz Piszczek's Polish League championship, in context there is information that his club achieved this and model probably did not deduct it means that their player also achieved this. 

## Why extractive question answering is not well suited for inflectional languages?

Extractive question answering struggles with inflectional languages due to the complexity of word forms and grammatical variations, making direct keyword matching less reliable. The multitude of word forms and varied grammatical structures in inflectional languages pose challenges for accurate question-to-text alignment in extractive systems.